In [10]:
# Some standard imports
import io
import numpy as np

from torch import nn
from models import TransformerNet
from utils import style_transform
import torch.onnx
from torch.autograd import Variable
from PIL import Image

In [8]:
model = 'checkpoints/bacon500_32000.pth'
image_path = 'images/content/zurich.jpeg'
name = 'bacon.onnx'

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = style_transform()

# Define model and load model checkpoint
torch_model = TransformerNet().to(device)
torch_model.load_state_dict(torch.load(model))
torch_model.eval()

batch_size = 1    # just a random number


In [11]:
# Input to the model
image_tensor = Variable(transform(Image.open(image_path))).to(device)
image_tensor = image_tensor.unsqueeze(0)

torch_out = torch_model(image_tensor)

# Export the model
torch.onnx.export(torch_model,               # model being run
                  image_tensor,                         # model input (or a tuple for multiple inputs)
                  name,   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})

/anaconda/envs/py35/lib/python3.5/site-packages/torch/onnx/utils.py:718: UserWarning: Provided key input for dynamic axes is not a valid input/output name
  warnings.warn("Provided key {} for dynamic axes is not a valid input/output name".format(key))
/anaconda/envs/py35/lib/python3.5/site-packages/torch/onnx/utils.py:718: UserWarning: Provided key output for dynamic axes is not a valid input/output name
  warnings.warn("Provided key {} for dynamic axes is not a valid input/output name".format(key))
